In [3]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt
%matplotlib inline

In [86]:
# read in all the words
with open('names.txt', 'r') as file:
    content = file.read()

content_lower = content.lower()

with open('names.txt', 'w') as file:
    file.write(content_lower)
    file.close()

In [87]:
# read in all the words
words = open('names.txt', 'r').read().splitlines()
words[:8]

['aaban',
 'aabharan',
 'aabhas',
 'aabhat',
 'aabheer',
 'abheer',
 'aabher',
 'aabi']

In [88]:
len(words)

53982

In [89]:
# build the vocabulary of characters and mappings to/from integers
chars = sorted(list(set(''.join(words)))) #get sorted list of letters from a to z
stoi = {s:i+1 for i,s in enumerate(chars)}
stoi['.'] = 0
itos = {i:s for s,i in stoi.items()}
print(itos)
len(itos)

{1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e', 6: 'f', 7: 'g', 8: 'h', 9: 'i', 10: 'j', 11: 'k', 12: 'l', 13: 'm', 14: 'n', 15: 'o', 16: 'p', 17: 'q', 18: 'r', 19: 's', 20: 't', 21: 'u', 22: 'v', 23: 'w', 24: 'x', 25: 'y', 26: 'z', 0: '.'}


27

In [90]:
# build the dataset
block_size = 3 # context length: how many characters do we take to predict the next one?

def build_dataset(words):  
  X, Y = [], []
  for w in words:

    #print(w)
    context = [0] * block_size
    for ch in w + '.':
      ix = stoi[ch]
      X.append(context)
      Y.append(ix)
      #print(''.join(itos[i] for i in context), '--->', itos[ix])
      context = context[1:] + [ix] # crop and append

  X = torch.tensor(X)
  Y = torch.tensor(Y)
  print(X.shape, Y.shape)
  return X, Y
X, Y = build_dataset(words)

torch.Size([488074, 3]) torch.Size([488074])


In [91]:
#now building training, testing and validation datasets
import random
random.seed(42)
random.shuffle(words)
n1 = int(0.8*len(words))
n2 = int(0.9*len(words))

Xtr, Ytr = build_dataset(words[:n1])
Xdev, Ydev = build_dataset(words[n1:n2])
Xte, Yte = build_dataset(words[n2:])

torch.Size([390591, 3]) torch.Size([390591])
torch.Size([48694, 3]) torch.Size([48694])
torch.Size([48789, 3]) torch.Size([48789])


In [92]:
C = torch.randn((27, 2))

In [93]:
emb = C[X]
emb.shape

torch.Size([488074, 3, 2])

In [94]:
W1 = torch.randn((6, 100))
b1 = torch.randn(100)

In [95]:
h = torch.tanh(emb.view(-1, 6) @ W1 + b1)

In [96]:
h

tensor([[-0.1057, -0.0737, -0.3409,  ..., -0.8047,  0.0429, -0.4071],
        [ 0.9694,  0.8355, -0.6682,  ...,  0.0972, -0.8793, -0.8468],
        [ 0.2732, -0.9958, -0.9424,  ..., -0.9672, -0.6095,  0.4632],
        ...,
        [ 0.9960,  0.9887,  0.4736,  ...,  0.9991, -0.9787, -0.9984],
        [-0.9583, -0.9991, -0.6222,  ..., -0.9967,  0.2632,  0.5954],
        [ 0.9987, -0.8833, -0.9990,  ...,  0.8902, -0.8808, -0.6524]])

In [97]:
h.shape

torch.Size([488074, 100])

In [98]:
W2 = torch.randn((100, 27))
b2 = torch.randn(27)

In [99]:
logits = h @ W2 + b2

In [100]:
logits.shape

torch.Size([488074, 27])

In [101]:
counts = logits.exp()

In [102]:
prob = counts / counts.sum(1, keepdims=True)

In [103]:
prob.shape

torch.Size([488074, 27])

In [105]:
loss = -prob[torch.arange(488074), Y].log().mean()
loss

tensor(16.7684)

In [106]:
Xtr.shape, Ytr.shape # dataset

(torch.Size([390591, 3]), torch.Size([390591]))

In [107]:
g = torch.Generator().manual_seed(2147483647) # for reproducibility
C = torch.randn((27, 10), generator=g)
W1 = torch.randn((30, 200), generator=g)
b1 = torch.randn(200, generator=g)
W2 = torch.randn((200, 27), generator=g)
b2 = torch.randn(27, generator=g)
parameters = [C, W1, b1, W2, b2]

In [108]:
sum(p.nelement() for p in parameters) # number of parameters in total

11897

In [109]:
for p in parameters:
  p.requires_grad = True

In [110]:
lre = torch.linspace(-3, 0, 1000)
lrs = 10**lre

In [111]:
lri = []
lossi = []
stepi = []

In [121]:
for i in range(10000):
  
  # minibatch construct
  ix = torch.randint(0, Xtr.shape[0], (32,))
  
  # forward pass
  emb = C[Xtr[ix]] # (32, 3, 10)
  h = torch.tanh(emb.view(-1, 30) @ W1 + b1) # (32, 200)
  logits = h @ W2 + b2 # (32, 27)
  loss = F.cross_entropy(logits, Ytr[ix])
  #print(loss.item())
  
  # backward pass
  for p in parameters:
    p.grad = None
  loss.backward()
  
  # update
  #lr = lrs[i]
  lr = 0.1 if i < 100000 else 0.01
  for p in parameters:
    p.data += -lr * p.grad

  # track stats
  #lri.append(lre[i])
  stepi.append(i)
  lossi.append(loss.log10().item())

print(loss.item())

1.471660852432251


In [122]:


# sample from the model
g = torch.Generator().manual_seed(2147483647 + 10)

for _ in range(50):
    
    out = []
    context = [0] * block_size # initialize with all ...
    while True:
      emb = C[torch.tensor([context])] # (1,block_size,d)
      h = torch.tanh(emb.view(1, -1) @ W1 + b1)
      logits = h @ W2 + b2
      probs = F.softmax(logits, dim=1)
      ix = torch.multinomial(probs, num_samples=1, generator=g).item()
      context = context[1:] + [ix]
      out.append(ix)
      if ix == 0:
        break
    
    print(''.join(itos[i] for i in out))

brikalmyan.
shen.
nihayalareshwarekgolek.
pishka.
thi.
pula.
pen.
dishayananarakalakshmaalin.
shubargahimiya.
vini.
pallilanthucksuthan.
dhruyallamehayamajayakshan.
hil.
sakya.
suhayakhlvan.
rithra.
puvikosh.
poodiryankoyohan.
priyashwathya.
hirayalini.
priya.
prashala.
ditarash.
priyohan.
bhitya.
dhaya.
vishmeelakethra.
praasha.
dumalarkicoothivalla.
ilbanahivarlen.
jashrayanikar.
pravaii.
shivoniya.
jyonyakshini.
yishan.
moosh.
priyonimeerthicallukeyushana.
coomilokkesh.
chu.
chanikshiniyhta.
moshayamalahadchan.
omayooshthan.
kshiyam.
koonkarunchelosh.
saini.
pin.
cinehanushayakashmaya.
divyaakhavathayuthanashyuduzh.
puniyalayuny.
than.
